In [4]:
import pyodbc
import numpy as np
import pandas as pd

In [5]:
# local db info
server = '123.0.0.1'
database = 'your_db_name'
username = 'your_username'
password = 'your_pass'
driver = '{ODBC Driver 18 for SQL Server}'

# ENCRYPT defaults to yes starting in ODBC Driver 18. It's good to always specify ENCRYPT=yes on the client side to avoid MITM attacks.
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';ENCRYPT=no;UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [6]:
data = pd.read_sql("SELECT TOP(100) * FROM master.dbo.job_run_cluster_analysis", cnxn)

In [7]:
query = """
SELECT DISTINCT 
	info1.job_run_date,
	info1.cluster_id,
	info1.vms_cost,
	info1.driver_node_type_id,
	info1.node_type_id,
	cts.worker_count,
	cts.cluster_type,
	cts.segment_start_time,
	cts.segment_end_time,
	cts.duration_sec
FROM
	(SELECT
		jrtc.subscription_id, 
		jrtc.workspace_id, 
		jrtc.job_id, 
		jrtc.task_id, 
		jrtc.cluster_id, 
		jrtc.job_run_date, 
		jrtc.vms_cost,
		JSON_VALUE(jrca.cluster_config, '$.driver_node_type_id') AS driver_node_type_id,
		JSON_VALUE(jrca.cluster_config, '$.node_type_id') AS node_type_id
	FROM 
		job_run_cluster_analysis jrca
	INNER JOIN
		job_run_task_cost jrtc
	ON
		jrca.db_job_id = jrtc.job_id
		AND jrca.cluster_id = jrtc.cluster_id 
		AND jrca.workspace_id = jrtc.workspace_id 
		AND jrca.subscription_id = jrtc.subscription_id 
		AND jrca.job_run_date = jrtc.job_run_date
	WHERE 
		jrtc.vms_cost IS NOT NULL
		AND jrca.cluster_config LIKE '%"availability":"SPOT%'
		AND	jrca.job_run_date >= '2024-01-01'
		AND jrca.job_run_date < '2024-02-01') info1
INNER JOIN
	cluster_timeline_segments cts
ON
	info1.subscription_id = cts.subscription_id 
	AND info1.workspace_id = cts.workspace_id 
	AND info1.cluster_id = cts.cluster_id 
	AND info1.job_run_date = cts.date
ORDER BY info1.job_run_date
"""

In [ ]:
df = pd.read_sql(query, cnxn)
df

# Another way using create_engine

In [2]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

In [2]:
# local db info

# server = '123.0.0.1'
server = "localhost"
database = 'your_db_name'
username = 'your_username'
password = 'your_pass'
driver = '{ODBC Driver 18 for SQL Server}'
port = "1433"


connection_string = f'DRIVER={driver};SERVER=tcp:{server};PORT=1433;DATABASE={database};'f'UID={username};PWD={password};TrustServerCertificate=YES;'
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url)

In [ ]:
with engine.begin() as conn:
    df = pd.read_sql(query, conn)

df.head()